In [ ]:
import os, sys, argparse

In [ ]:
import os, sys, argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import logging
import allel


#import custom functions

#sys.path.append('./')

import vcfReadAndParse
import Fst_diploids
#import initialize_calculation_DataFrame
import parameter_estimate_on_neutral_FST_distribution
#import calculate_pvalue_forAllLoci
import visualization
import ld_prune

In [ ]:
os.chdir("/Users/zhihai/box/CABBI08192021/downstream_analyses_Pan_1k/gwas/gwas_for_all/blinkC/exploreBlinkCBurstPvalue")

In [ ]:
vcf_file_path = "1056_sorghum_imputed_SNP_maf005_biallelic_NumChr_ModiID_snpwin100slidewin50ld08_1.vcf.gz"

In [ ]:
position_info, genotype_info = vcfReadAndParse.read_vcf_file(vcf_file_path)

#format conversion
binary_data = vcfReadAndParse.process_genotype(genotype_info)

G_matrix = pd.DataFrame(binary_data).astype(int)

In [ ]:
genotype_matrix = G_matrix

In [ ]:
correlation_matrix = np.corrcoef(genotype_matrix)

: 

In [ ]:
G_matrix

In [ ]:
correlation_matrix = np.corrcoef(genotype_matrix)

In [ ]:
device